# 第六章：数据编码和处理
这一章主要讨论使用Python处理各种不同方式编码的数据，比如CSV文件，JSON，XML和二进制包装记录。

和数据结构那一章不同的是，这章不会讨论特殊的算法问题，而是关注于怎样获取和存储这些格式的数据。

## 6.1 读写CSV数据

* 问题

你想读写一个CSV格式的文件。

* 解决方案

对于大多数的CSV格式的数据读写问题，都可以使用 csv 库。 例如：假设你在一个名叫 `stocks.csv` 文件中有一些股票市场数据，就像这样：

In [18]:
# 读取csv数据，转为元组序列
import csv

with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        print(row)
        print(isinstance(row, list))

['AA', '39.48', '6/11/2007', '9:36am', '-0.18', '181800']
True
['AIG', '71.38', '6/11/2007', '9:36am', '-0.15', '195500']
True
['AXP', '62.58', '6/11/2007', '9:36am', '-0.46', '935000']
True
['BA', '98.31', '6/11/2007', '9:36am', '+0.12', '104800']
True
['C', '53.08', '6/11/2007', '9:36am', '-0.25', '360900']
True
['CAT', '78.29', '6/11/2007', '9:36am', '-0.23', '225400']
True


In [21]:
row[0], row[1], row[2], row[3], row[4], row[5]

('CAT', '78.29', '6/11/2007', '9:36am', '-0.23', '225400')

在上面的代码中，`row` 会是一个列表。

因此，为了访问某个字段，你需要使用下标。

如 `row[0]` 访问 `Symbol`的值，`row[4]` 访问 `Change` 的值。

In [24]:
# 使用下标访问通常会引起混淆，使用 命名元组来索引数据。

from collections import namedtuple

with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headings = next(f_csv)  # 列名
    Row = namedtuple('Row', headings)
    for r in f_csv:
        # r is list
        row = Row(*r)
        print(row)
        print(row.Symbol)  # 使用列名索引数据

Row(Symbol='AA', Price='39.48', Date='6/11/2007', Time='9:36am', Change='-0.18', Volume='181800')
AA
Row(Symbol='AIG', Price='71.38', Date='6/11/2007', Time='9:36am', Change='-0.15', Volume='195500')
AIG
Row(Symbol='AXP', Price='62.58', Date='6/11/2007', Time='9:36am', Change='-0.46', Volume='935000')
AXP
Row(Symbol='BA', Price='98.31', Date='6/11/2007', Time='9:36am', Change='+0.12', Volume='104800')
BA
Row(Symbol='C', Price='53.08', Date='6/11/2007', Time='9:36am', Change='-0.25', Volume='360900')
C
Row(Symbol='CAT', Price='78.29', Date='6/11/2007', Time='9:36am', Change='-0.23', Volume='225400')
CAT


上述代码允许使用列名如 `row.Symbol` 和 `row.Change` 代替下标访问数据。

需要注意的是：这个只有在 **列名是合法的Python标识符** 的时候才生效。

如果不是的话，你可能需要修改下原始的列名(如将非标识符字符替换成下划线之类的)。

另外一个选择就是**将数据读取到一个字典序列**中去。可以这样做：

In [11]:
import csv

with open('stocks.csv') as f:
    f_csv = csv.DictReader(f)

    for row in f_csv:
        print(row)
        print(row['Symbol'])

OrderedDict([('Symbol', 'AA'), ('Price', '39.48'), ('Date', '6/11/2007'), ('Time', '9:36am'), ('Change', '-0.18'), ('Volume', '181800')])
AA
OrderedDict([('Symbol', 'AIG'), ('Price', '71.38'), ('Date', '6/11/2007'), ('Time', '9:36am'), ('Change', '-0.15'), ('Volume', '195500')])
AIG
OrderedDict([('Symbol', 'AXP'), ('Price', '62.58'), ('Date', '6/11/2007'), ('Time', '9:36am'), ('Change', '-0.46'), ('Volume', '935000')])
AXP


在这个版本中，你可以使用列名去访问每一行的数据了。比如，`row['Symbol']` 或者 `row['Change']`


为了写入CSV数据，你仍然可以使用csv模块，不过这时候先创建一个 `writer` 对象。例如:

In [27]:
headers = ['Symbol','Price','Date','Time','Change','Volume']
rows = [('AA', 39.48, '6/11/2007', '9:36am', -0.18, 181800),
        ('AIG', 71.38, '6/11/2007', '9:36am', -0.15, 195500),
        ('AXP', 62.58, '6/11/2007', '9:36am', -0.46, 935000)
        ]

with open('stocks.csv','w') as f:
    f_csv = csv.writer(f)
    f_csv.writerow(headers)
    f_csv.writerows(rows)

In [14]:
# 如果你有一个字典序列的数据
headers = ['Symbol', 'Price', 'Date', 'Time', 'Change', 'Volume']

rows = [{'Symbol':'AA', 'Price':39.48, 'Date':'6/11/2007',
        'Time':'9:36am', 'Change':-0.18, 'Volume':181800},
        {'Symbol':'AIG', 'Price': 71.38, 'Date':'6/11/2007',
        'Time':'9:36am', 'Change':-0.15, 'Volume': 195500},
        {'Symbol':'AXP', 'Price': 62.58, 'Date':'6/11/2007',
        'Time':'9:36am', 'Change':-0.46, 'Volume': 935000},
        ]

with open('stocks.csv','w') as f:
    f_csv = csv.DictWriter(f, headers)
    f_csv.writeheader()
    f_csv.writerows(rows)

* 讨论

你应该优先选择 `csv` 模块分割或解析CSV数据。例如，你可能会像编写类似下面这样的代码：

In [29]:
with open('stocks.csv') as f:
    for line in f:
        row = line.split(',')
        print(row)

['Symbol', 'Price', 'Date', 'Time', 'Change', 'Volume\n']
['"AA"', '39.48', '"6/11/2007"', '"9:36am"', '-0.18', '181800\n']
['"AIG"', '71.38', '"6/11/2007"', '"9:36am"', '-0.15', '195500\n']
['"AXP"', '62.58', '"6/11/2007"', '"9:36am"', '-0.46', '935000\n']
['"B', 'A"', '98.31', '"6/11/2007"', '"9:36am"', '+0.12', '104800\n']
['"C"', '53.08', '"6/11/2007"', '"9:36am"', '-0.25', '360900\n']
['"CAT"', '78.29', '"6/11/2007"', '"9:36am"', '-0.23', '225400']


使用这种方式的一个缺点就是你仍然需要去处理一些棘手的细节问题。
* 如果某些字段值被引号包围，你不得不去除这些引号。
* 如果一个被引号包围的字段碰巧含有一个逗号，那么程序就会因为产生一个错误大小的行而出错。

默认情况下，csv 库可识别Microsoft Excel所使用的CSV编码规则。这也是最常见的形式，并且也会给你带来最好的兼容性。

然而，如果你查看csv的文档，就会发现有很多种方法将它应用到其他编码格式上(如修改分割字符等)。 例如，如果你想读取以tab分割的数据，可以这样做：

```python
# Example of reading tab-separated values
with open('stock.tsv') as f:
    f_tsv = csv.reader(f, delimiter='\t')
    for row in f_tsv:
        # Process row
        print(row)
```

如果你正在读取CSV数据并将它们转换为 命名元组，需要注意对列名进行合法性认证。

例如，一个CSV格式文件有一个包含非法标识符的列头行，类似下面这样：
```
Street Address,Num-Premises,Latitude,Longitude 5412 N CLARK,10,41.980262,-87.668452
```

这样最终会导致在创建一个命名元组时产生一个 ValueError 异常而失败。

为了解决这问题，你可能不得不先去修正列标题。例如，可以像下面这样在非法标识符上使用一个正则表达式替换：

```python
import re
with open('stock.csv') as f:
    f_csv = csv.reader(f)
    headers = [ re.sub('[^a-zA-Z_]', '_', h) for h in next(f_csv) ]
    Row = namedtuple('Row', headers)
    for r in f_csv:
        row = Row(*r)
        # Process row
        ...
```

还有重要的一点需要强调的是，csv产生的数据都是**字符串类型**的，它不会做任何其他类型的转换。

如果你需要做这样的类型转换，你必须自己手动去实现。下面是一个在CSV数据上执行其他类型转换的例子：
```python
col_types = [str, float, str, str, float, int]
with open('stocks.csv') as f:
    f_csv = csv.reader(f)
    headers = next(f_csv)
    for row in f_csv:
        # Apply conversions to the row items
        row = tuple(convert(value) for convert, value in zip(col_types, row))
```

另外，下面是一个转换字典中特定字段的例子：

In [16]:
print('Reading as dicts with type conversion')

field_types = [ ('Price', float),
                ('Change', float),
                ('Volume', int) ]

with open('stocks.csv') as f:
    for row in csv.DictReader(f):
        row.update((key, conversion(row[key])) for key, conversion in field_types)
        print(row)

Reading as dicts with type conversion
OrderedDict([('Symbol', 'AA'), ('Price', 39.48), ('Date', '6/11/2007'), ('Time', '9:36am'), ('Change', -0.18), ('Volume', 181800)])
OrderedDict([('Symbol', 'AIG'), ('Price', 71.38), ('Date', '6/11/2007'), ('Time', '9:36am'), ('Change', -0.15), ('Volume', 195500)])
OrderedDict([('Symbol', 'AXP'), ('Price', 62.58), ('Date', '6/11/2007'), ('Time', '9:36am'), ('Change', -0.46), ('Volume', 935000)])


通常来讲，你可能并不想过多去考虑这些转换问题。在实际情况中，CSV文件都或多或少有些缺失的数据，被破坏的数据以及其它一些让转换失败的问题。

因此，除非你的数据确实有保障是准确无误的，否则你必须考虑这些问题(你可能需要增加合适的错误处理机制)。

最后，如果你读取CSV数据的目的是做数据分析和统计的话，你可能需要看一看 `Pandas` 包。

Pandas 包含了一个非常方便的函数叫 `pandas.read_csv()`，它可以加载CSV数据到一个 `DataFrame` 对象中去。 然后利用这个对象你就可以生成各种形式的统计、过滤数据以及执行其他高级操作了。在6.13小节中会有这样一个例子。

## 6.2 读写JSON数据

* 问题

读写JSON(JavaScript Object Notation)编码格式的数据。

* 解决方案

`json` 模块提供了一种很简单的方式来编码和解码JSON数据。其中两个主要的函数是 `json.dumps()` 和 `json.loads()`，

要比其他序列化函数库如pickle的接口少得多。 下面演示如何将一个Python数据结构转换为JSON：

In [30]:
import json

data = {
    'name': 'ACME',
    'shares': 100,
    'price': 542.23
}

json_str = json.dumps(data)

下面演示如何将一个JSON编码的字符串转换回一个Python数据结构：

In [33]:
data = json.loads(json_str)
data

{'name': 'ACME', 'shares': 100, 'price': 542.23}

如果你要处理的是文件而不是字符串，你可以使用 `json.dump()` 和 `json.load()` 来编码和解码JSON数据。例如：

In [34]:
# Writing JSON data
with open('data.json', 'w') as f:
    json.dump(data, f)

# Reading data back
with open('data.json', 'r') as f:
    data = json.load(f)

* 讨论

JSON编码支持的基本数据类型为 `None`，`bool`，`int`，`float` 和 `str`，以及包含这些类型数据的 `lists`，`tuples` 和 `dictionaries`。

对于`dictionaries`，`keys` 需要是字符串类型（字典中任何非字符串类型的key在编码时会先转换为字符串）。

为了遵循JSON规范，你应该只编码Python的 `lists` 和 `dictionaries`。 而且，在web应用程序中，顶层对象被编码为一个字典是一个标准做法。

JSON编码的格式对于Python语法而已几乎是完全一样的，除了一些小的差异之外。比如，True会被映射为true，False被映射为false，而None会被映射为null。

下面是一个例子，演示了编码后的字符串效果：

In [36]:
json.dumps(False)

d = {
    'a': True,
    'b': 'Hello',
    'c': None
    }

json.dumps(d)

'{"a": true, "b": "Hello", "c": null}'

如果你试着去检查JSON解码后的数据，你通常很难通过简单的打印来确定它的结构，特别是当数据的嵌套结构层次很深或者包含大量的字段时。

为了解决这个问题，可以考虑使用 `pprint` 模块的 `pprint()` 函数来代替普通的 `print()` 函数。

它会按照key的字母顺序并以一种更加美观的方式输出。 下面是一个演示如何漂亮的打印输出Twitter上搜索结果的例子：

In [39]:
from urllib.request import urlopen
import json


u = urlopen('https://mbd.baidu.com/newspage/data/landingsuper?context=%7B%22nid%22%3A%22news_9398506483755601800%22%7D&n_type=-1&p_from=-1')
resp = json.loads(u.read().decode('utf-8'))

from pprint import pprint
pprint(resp)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

一般来讲，JSON解码会根据提供的数据创建 `dicts` 或`lists`。

如果你想要创建其他类型的对象，可以给 `json.loads()` 传递 `object_pairs_hook` 或 `object_hook` 参数。

例如，下面是演示如何解码JSON数据并在一个 `OrderedDict` 中保留其顺序的例子：

In [42]:
s = '{"name": "ACME", "shares": 50, "price": 490.1}'
from collections import OrderedDict
data = json.loads(s, object_pairs_hook=OrderedDict)
data

OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])

下面是如何将一个JSON字典转换为一个Python对象例子：

In [43]:
class JSONObject:
    def __init__(self, d):
        self.__dict__ = d

In [44]:
data = json.loads(s, object_hook=JSONObject)
data.name

'ACME'

In [45]:
data.shares

50

In [46]:
data.price

490.1

JSON解码后的字典作为一个单个参数传递给 `__init__()`。然后，你就可以随意使用它了，比如作为一个实例字典来直接使用它。

在编码JSON的时候，还有一些选项很有用。 如果你想获得漂亮的格式化字符串后输出，可以使用 `json.dumps()` 的 `indent` 参数。 它会使得输出和pprint()函数效果类似。比如：

In [51]:
s = '{"name": "ACME", "shares": 50, "price": 490.1}'
from collections import OrderedDict
data = json.loads(s, object_pairs_hook=OrderedDict)
print(json.dumps(data))

{"name": "ACME", "shares": 50, "price": 490.1}


In [53]:
print(json.dumps(data, indent=4))

{
    "name": "ACME",
    "shares": 50,
    "price": 490.1
}


对象实例通常并不是JSON可序列化的。例如：

In [57]:
class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y


p = Point(2, 3)
json.dumps(p)

TypeError: Object of type 'Point' is not JSON serializable

如果你想序列化对象实例，你可以提供一个函数，它的输入是一个实例，返回一个可序列化的字典。例如：

In [60]:
def serialize_instance(obj):
    d = { '__classname__' : type(obj).__name__ }
    d.update(vars(obj))
    return d

In [61]:
# 如果你想反过来获取这个实例，可以这样做：
# Dictionary mapping names to known classes
classes = {
    'Point' : Point
}

def unserialize_object(d):
    clsname = d.pop('__classname__', None)
    if clsname:
        cls = classes[clsname]
        obj = cls.__new__(cls) # Make instance without calling __init__
        for key, value in d.items():
            setattr(obj, key, value)
        return obj
    else:
        return d

In [62]:
# 下面是如何使用这些函数的例子：
p = Point(2, 3)
s = json.dumps(p, default=serialize_instance)
s


'{"__classname__": "Point", "x": 2, "y": 3}'

In [63]:
a = json.loads(s, object_hook=unserialize_object)
a

In [64]:
a.x

2

In [65]:
a.y

3

json 模块还有很多其他选项来控制更低级别的数字、特殊值如NaN等的解析。 可以参考官方文档获取更多细节。

## 6.3 解析简单的XML数据

* 问题

你想从一个简单的XML文档中提取数据。

* 解决方案

可以使用 `xml.etree.ElementTree` 模块从简单的XML文档中提取数据。为了演示，假设你想解析 Planet Python上的RSS源。下面是相应的代码：

In [66]:
from urllib.request import urlopen
from xml.etree.ElementTree import parse

# Download the RSS feed an dparse it
u = urlopen('http://planet.python.org/rss20.xml')
doc = parse(u)

In [67]:
# Extract and output tags of interest
for item in doc.iterfind('channel/item'):
    title = item.findtext('title')
    date = item.findtext('pubDate')
    link = item.findtext('link')

    print(title)
    print(data)
    print(link)

Real Python: Build and Submit HTML Forms With Django – Part 4
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
https://realpython.com/django-social-forms-4/
Python for Beginners: Create Decorators Using Classes in Python
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
https://www.pythonforbeginners.com/basics/create-decorators-using-classes-in-python
Quansight Labs Blog: IPython 8.0, Lessons learned maintaining software
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
https://labs.quansight.org/blog/2022/01/ipython-8.0-lessons-learned-maintaining-software/
Python Circle: Django vs Node.js: Which One Is Better for Web Development?
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
https://pythoncircle.com/post/757/django-vs-nodejs-which-one-is-better-for-web-development/
Kay Hayen: Nuitka Release 0.6.19
OrderedDict([('name', 'ACME'), ('shares', 50), ('price', 490.1)])
https://nuitka.net/posts/nuitka-release-0619.html
Tes

* 讨论

在很多应用程序中处理XML编码格式的数据是很常见的。不仅因为XML在Internet上面已经被广泛应用于数据交换，同时它也是一种存储应用程序数据的常用格式(比如字处理，音乐库等)。

在很多情况下，当使用XML来仅仅存储数据的时候，对应的文档结构非常紧凑并且直观。 例如，上面例子中的RSS订阅源类似于下面的格式：
```xml
<?xml version="1.0"?>
<rss version="2.0" xmlns:dc="http://purl.org/dc/elements/1.1/">
    <channel>
        <title>Planet Python</title>
        <link>http://planet.python.org/</link>
        <language>en</language>
        <description>Planet Python - http://planet.python.org/</description>
        <item>
            <title>Steve Holden: Python for Data Analysis</title>
            <guid>http://holdenweb.blogspot.com/...-data-analysis.html</guid>
            <link>http://holdenweb.blogspot.com/...-data-analysis.html</link>
            <description>...</description>
            <pubDate>Mon, 19 Nov 2012 02:13:51 +0000</pubDate>
        </item>
        <item>
            <title>Vasudev Ram: The Python Data model (for v2 and v3)</title>
            <guid>http://jugad2.blogspot.com/...-data-model.html</guid>
            <link>http://jugad2.blogspot.com/...-data-model.html</link>
            <description>...</description>
            <pubDate>Sun, 18 Nov 2012 22:06:47 +0000</pubDate>
        </item>
        <item>
            <title>Python Diary: Been playing around with Object Databases</title>
            <guid>http://www.pythondiary.com/...-object-databases.html</guid>
            <link>http://www.pythondiary.com/...-object-databases.html</link>
            <description>...</description>
            <pubDate>Sun, 18 Nov 2012 20:40:29 +0000</pubDate>
        </item>
        ...
    </channel>
</rss>
```

`xml.etree.ElementTree.parse()` 函数解析整个XML文档并将其转换成一个文档对象。

然后，你就能使用 `find()`、`iterfind()` 和 `findtext()` 等方法来搜索特定的XML元素了。

这些函数的参数就是某个指定的标签名，例如 `channel/item` 或 `title`。

每次指定某个标签时，你需要遍历整个文档结构。每次搜索操作会从一个起始元素开始进行。同样，每次操作所指定的标签名也是起始元素的相对路径。

例如，执行 `doc.iterfind('channel/item')` 来搜索所有在 `channel` 元素下面的 `item` 元素。

`doc` 代表文档的最顶层(也就是第一级的 rss 元素)。 然后接下来的调用 `item.findtext()` 会从已找到的 `item` 元素位置开始搜索。

`ElementTree` 模块中的每个元素有一些重要的属性和方法，在解析的时候非常有用。

`tag` 属性包含了标签的名字，`text` 属性包含了内部的文本，而 `get()` 方法能获取属性值。例如：

In [68]:
doc

In [70]:
e = doc.find('channel/title')
e

<Element 'title' at 0x0000028DF66490E8>

In [71]:
e.tag

'title'

In [72]:
e.text

'Planet Python'

In [73]:
e.get('some_attribute')

有一点要强调的是 `xml.etree.ElementTree` 并不是XML解析的唯一方法。

对于更高级的应用程序，你需要考虑使用 `lxml`。 它使用了和ElementTree同样的编程接口，因此上面的例子同样也适用于lxml。

你只需要将刚开始的import语句换成 `from lxml.etree import parse` 就行了。

`lxml` 完全遵循XML标准，并且速度也非常快，同时还支持验证，XSLT，和XPath等特性。